In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [3]:
df=pd.read_csv("Resources/lending_data.csv")
df

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0
...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600,1
77532,17700.0,10.662,80900,0.629172,11,2,50900,1
77533,17600.0,10.595,80300,0.626401,11,2,50300,1
77534,16300.0,10.068,75300,0.601594,10,2,45300,1


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [7]:
y= df['loan_status']
X=df.drop(columns=['loan_status'])

In [8]:
y

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [9]:
X

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000
...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600
77532,17700.0,10.662,80900,0.629172,11,2,50900
77533,17600.0,10.595,80300,0.626401,11,2,50300
77534,16300.0,10.068,75300,0.601594,10,2,45300


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [10]:
tb=y.value_counts()
tb

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [13]:
from sklearn.linear_model import LogisticRegression
lr_mod= LogisticRegression(random_state=1)
lr_mod.fit(X_train, y_train)


LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [14]:
y_pred= lr_mod.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [19]:
b_acc= balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score:", b_acc)
cm= confusion_matrix(y_test, y_pred)
cmdf= pd.DataFrame(cm, columns=['Predicted 0', 'Predicted 1'], index=['True 0','True 1'])
class_report= classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

Balanced Accuracy Score: 0.9521352751368186
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15001
           1       0.86      0.91      0.88       507

    accuracy                           0.99     15508
   macro avg       0.93      0.95      0.94     15508
weighted avg       0.99      0.99      0.99     15508



In [18]:
print("Confusion Matrix:")
cmdf

Confusion Matrix:


,Predicted 0,Predicted 1
True 0,14926,75
True 1,46,461


### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** I'd say its pretty darn accurate! It is only allowing for about 1% to approximately 5% room for error. This model predicts very well for this dataset.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [20]:
from imblearn.over_sampling import RandomOverSampler
osamp= RandomOverSampler(random_state=1)
X_train_rs, y_train_rs= osamp.fit_resample(X_train, y_train)


In [21]:
dvc= y_train_rs.value_counts()
print(dvc)

0    60035
1    60035
Name: loan_status, dtype: int64


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [23]:
lr_mod_rs= LogisticRegression(random_state=1)
lr_mod_rs.fit(X_train_rs, y_train_rs)
y_pred_rs= lr_mod_rs.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [25]:
b_acc_rs= balanced_accuracy_score(y_test, y_pred_rs)
print("Balanced Accuracy Score for resampled data:", b_acc_rs)
cmrs= confusion_matrix(y_test, y_pred_rs)
cmrsdf= pd.DataFrame(cmrs, columns=['Predicted 0', 'Predicted 1'], index=['True 0','True 1'])
class_report_rs= classification_report(y_test, y_pred_rs)
print("Classification Report for resampled data:")
print(class_report_rs)

Balanced Accuracy Score for resampled data: 0.9941749445500477
Classification Report for resampled data:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     15001
           1       0.85      0.99      0.92       507

    accuracy                           0.99     15508
   macro avg       0.93      0.99      0.96     15508
weighted avg       1.00      0.99      0.99     15508



In [26]:
print("Confusion Matrix for resampled data:")
cmrsdf

Confusion Matrix for resampled data:


,Predicted 0,Predicted 1
True 0,14915,86
True 1,3,504


### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:* This resampled model is showing incredible accuracy for this dataset. There is ONLY 1% margin for error for predicting both healthy loans and high risk loans. The original model yielded a 95% balanced accuracy score while the resampled yielded 99% balanced accuracy. The original did exceptionally well, but the resampled model 'sharpened our arrow shot'. The original model had a small percent of high-risk cases that were not detected (about 9%). The resampled model is simply more sensitive to high-risk cases by way of oversampling, ultimately reducing false negatives. 

